In [ ]:
# pip install pygame
import pygame
import random
from queue import PriorityQueue
import time
from pygame.locals import *

#Global variables
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 800

GRID_SIZE = 8
TILE_SIZE = SCREEN_WIDTH // GRID_SIZE
GREEN = (0, 255, 0)
PURPLE = (128, 0, 128)
RED = (255, 0, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

class Player:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.radius = 10

    def draw(self,screen):
        pygame.draw.circle(screen, RED, (self.x*TILE_SIZE + TILE_SIZE//2, (GRID_SIZE-1-self.y)*TILE_SIZE + TILE_SIZE//2), self.radius)

class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.edges = {}
        self.isAnimal = False
        self.obstacleDirection = None
        self.heuristic_value = None

    def connect(self, node, weight):
        self.edges[node] = weight

    def draw(self, screen, color, border_color):
        if self.isAnimal:
            animal_image = pygame.image.load('animal.png')
            animal_image = pygame.transform.scale(animal_image, (TILE_SIZE, TILE_SIZE))
            screen.blit(animal_image, (self.x*TILE_SIZE, (GRID_SIZE-1-self.y)*TILE_SIZE))
        elif self.obstacleDirection is not None:
            obstacle_image = pygame.image.load('obstacle.jpg')
            obstacle_image = pygame.transform.scale(obstacle_image, (TILE_SIZE, TILE_SIZE))
            if self.obstacleDirection == 'up':
                obstacle_image = pygame.transform.rotate(obstacle_image, 90)
            elif self.obstacleDirection == 'down':
                obstacle_image = pygame.transform.rotate(obstacle_image, 270)
            elif self.obstacleDirection == 'left':
                obstacle_image = pygame.transform.rotate(obstacle_image, 180)
            screen.blit(obstacle_image, (self.x*TILE_SIZE, (GRID_SIZE-1-self.y)*TILE_SIZE))
        else:
            rect = pygame.Rect(self.x*TILE_SIZE, (GRID_SIZE-1-self.y)*TILE_SIZE, TILE_SIZE, TILE_SIZE)
            pygame.draw.rect(screen, color, rect)
        pygame.draw.rect(screen, border_color, pygame.Rect(self.x*TILE_SIZE, (GRID_SIZE-1-self.y)*TILE_SIZE, TILE_SIZE, TILE_SIZE), 1)

    def draw_coordinates(self, screen):
        font = pygame.font.Font(None, 24)
        text = font.render(f'({self.x},{self.y})', True, (255, 255, 255))
        screen.blit(text, (self.x*TILE_SIZE, (GRID_SIZE-1-self.y)*TILE_SIZE))
    def __lt__(self, other):
        return self.x < other.x if self.y == other.y else self.y < other.y
class Graph:
    def __init__(self, width, height, animals, obstacles):
        self.nodes = [[Node(x, y) for y in range(height)] for x in range(width)]
        self.width = width
        self.height = height
        for x in range(width):
            for y in range(height):
                weight = 1
                if x > 0:
                    self.nodes[x][y].connect(self.nodes[x - 1][y], weight)
                if x < width - 1:
                    self.nodes[x][y].connect(self.nodes[x + 1][y], weight)
                if y > 0:
                    self.nodes[x][y].connect(self.nodes[x][y - 1], weight)
                if y < height - 1:
                    self.nodes[x][y].connect(self.nodes[x][y + 1], weight)

        for animal in animals:
            self.nodes[animal[0]][animal[1]].isAnimal = True

        for obstacle in obstacles:
            cost = random.randint(2, 4)
            self.nodes[obstacle[0][0]][obstacle[0][1]].edges[self.nodes[obstacle[1][0]][obstacle[1][1]]] = cost
            self.nodes[obstacle[1][0]][obstacle[1][1]].edges[self.nodes[obstacle[0][0]][obstacle[0][1]]] = cost
            self.nodes[obstacle[0][0]][obstacle[0][1]].obstacleDirection = obstacle[2]
        for x in range(width):
            for y in range(height):
                self.nodes[x][y].heuristic_value = abs(x - (width-1)) + abs(y - (height-1))
        

    def draw(self, screen):
        for x in range(self.width):
            for y in range(self.height):
                if self.nodes[x][y].isAnimal:
                    self.nodes[x][y].draw(screen, BLACK, BLACK)
                elif (x, y) == (0, 0):
                    self.nodes[x][y].draw(screen, PURPLE, BLACK)
                elif (x, y) == (self.width-1, self.height-1):
                    self.nodes[x][y].draw(screen, RED, BLACK)
                else:
                    self.nodes[x][y].draw(screen, WHITE, BLACK)
                self.nodes[x][y].draw_coordinates(screen)
    def UCS(self, start, goal):
        frontier = PriorityQueue()
        frontier.put(start, 0)
        came_from = {}
        cost_so_far = {}
        came_from[start] = None
        cost_so_far[start] = 0
        while not frontier.empty():
            current = frontier.get()
            if current == goal:
                break
            for next_node, weight in current.edges.items():
                new_cost = cost_so_far[current] + weight
                if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                    cost_so_far[next_node] = new_cost
                    priority = new_cost
                    frontier.put(next_node, priority)
                    came_from[next_node] = current
        current = goal
        path = []
        while current is not None:
            path.append(current)
            current = came_from[current]
        path.reverse()
        total_cost = 0
        return path, total_cost
    def Astar(self, start, goal):
        frontier = PriorityQueue()
        frontier.put(start, 0)
        came_from = {}
        cost_so_far = {}
        came_from[start] = None
        cost_so_far[start] = 0
        total_cost = 0
        while not frontier.empty():
            current = frontier.get()
            if current == goal:
                break
            for next_node, weight in current.edges.items():
                if next_node.isAnimal:
                    random_dice = random.randint(0, 10)
                    if random_dice in [1,2]:
                        random_weight = random.randint(1, 3)
                        weight = random_weight
                    elif random_dice in [3,4,5,6,7,8,9,10]:
                        random_weight = random.randint(2,4)
                        weight = random_weight
                new_cost = cost_so_far[current] + weight + next_node.heuristic_value
                if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                    cost_so_far[next_node] = new_cost
                    priority = new_cost
                    frontier.put(next_node, priority)
                    came_from[next_node] = current
        current = goal
        path = []
        while current is not None:
            path.append(current)
            current = came_from[current]
        path.reverse()
        return path, total_cost
def showAstar(graph, start_node, goal_node, screen, player):
    path,total_cost = graph.Astar(start_node, goal_node)
    for i in range(len(path)-1):
        if path[i].isAnimal:
            random_dice = random.randint(0, 10)
            if random_dice in [1,2]:
                random_weight = random.randint(1, 3)
                total_cost += random_weight
            elif random_dice in [3,4,5,6,7,8,9,10]:
                random_weight = random.randint(2,4)
                total_cost += random_weight
        total_cost += path[i].edges[path[i+1]]
    visualize_path(path, screen, player,graph,total_cost)

def showUCS(graph, start_node, goal_node, screen, player):
    path,total_cost = graph.UCS(start_node, goal_node)
    total_cost = 0
    for i in range(len(path)-1):
        if path[i].isAnimal:
            random_dice = random.randint(0, 10)
            if random_dice in [1,2]:
                random_weight = random.randint(1, 3)
                total_cost += random_weight
            elif random_dice in [3,4,5,6,7,8,9,10]:
                random_weight = random.randint(2,4)
                total_cost += random_weight
        total_cost += path[i].edges[path[i+1]]
    visualize_path(path, screen, player,graph,total_cost)

def visualize_path(path, screen, player,graph,total_cost):
    running = True
    path_index = 0
    font = pygame.font.Font(None, 36)
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        screen.fill((255, 255, 255))
        graph.draw(screen)
        #Calculate the total cost for the path
        if path_index < len(path):
            player.x = path[path_index].x
            player.y = path[path_index].y
            path_index += 1
            time.sleep(0.2)
        elif path_index == len(path):
            pygame.draw.rect(screen, (255,255,255), pygame.Rect(0, SCREEN_HEIGHT // 2 - 20, SCREEN_WIDTH, 40))
            cost_text = font.render('Total cost: ' + str(total_cost), True, (0, 0, 0))
            screen.blit(cost_text, (SCREEN_WIDTH // 2 - cost_text.get_width() // 2, SCREEN_HEIGHT // 2 - cost_text.get_height() // 2))
        player.draw(screen)
        pygame.display.flip()

    pygame.quit()
    
    


def main():
    pygame.init()

    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    font = pygame.font.Font(None, 36)
    background = pygame.image.load('background.jpg')
    button_width, button_height = 200, 50
    astar_button = pygame.Rect(SCREEN_WIDTH // 2 - button_width // 2, SCREEN_HEIGHT // 2, button_width, button_height)
    ucs_button = pygame.Rect(SCREEN_WIDTH // 2 - button_width // 2, SCREEN_HEIGHT // 2 + 60, button_width, button_height)
    button_color = (0, 255, 0)
    hover_color = (255, 0, 0)

    animals = []
    obstacles = []
    for i in range(random.randint(10, 15)):
        random_x = random.randint(0, GRID_SIZE-1)
        random_y = random.randint(0, GRID_SIZE-1)
        if (random_x,random_y) == (0,0) or (random_x,random_y) == (GRID_SIZE-1,GRID_SIZE-1):
            continue
        animals.append((random_x, random_y))
    for i in range(random.randint(3, 5)):
        x = random.randint(0, GRID_SIZE-1)
        y = random.randint(0, GRID_SIZE-1)
        if (x,y) == (0,0) or (x,y) == (GRID_SIZE-1,GRID_SIZE-1):
            continue
        direction = random.choice(["up", "down", "left", "right"])
        if x == 0 and direction == "left":
            direction = "right"
        if x == GRID_SIZE-1 and direction == "right":
            direction = "left"
        if y == 0 and direction == "up":
            direction = "down"
        if y == GRID_SIZE-1 and direction == "down":
            direction = "up"
        if direction == "up":
            obstacles.append(((x, y), (x, y - 1),'up'))
        elif direction == "down":
            obstacles.append(((x, y), (x, y + 1),'down'))
        elif direction == "left":
            obstacles.append(((x, y), (x - 1, y),'left'))
        elif direction == "right":
            obstacles.append(((x, y), (x + 1, y),'right'))


    graph = Graph(GRID_SIZE, GRID_SIZE, animals, obstacles)

    player = Player(0, 0)


    start_node = graph.nodes[0][0]
    goal_node = graph.nodes[GRID_SIZE-1][GRID_SIZE-1]

    running = True
    while running:
        mouse_pos = pygame.mouse.get_pos() 
        screen.blit(background, (0, 0))
        for event in pygame.event.get():
            if event.type == QUIT:
                running = False
            elif event.type == MOUSEBUTTONDOWN:
                if astar_button.collidepoint(event.pos):
                    showAstar(graph, start_node, goal_node, screen, player)
                elif ucs_button.collidepoint(event.pos):
                    showUCS(graph, start_node, goal_node, screen, player)

        if astar_button.collidepoint(mouse_pos):
            pygame.draw.rect(screen, hover_color, astar_button) 
        else:
            pygame.draw.rect(screen, button_color, astar_button) 

        if ucs_button.collidepoint(mouse_pos):
            pygame.draw.rect(screen, hover_color, ucs_button)  
        else:
            pygame.draw.rect(screen, button_color, ucs_button) 

        astar_text = font.render('A*', True, (0, 0, 0))
        ucs_text = font.render('UCS', True, (0, 0, 0))
        screen.blit(astar_text, (astar_button.x + 50, astar_button.y + 10))
        screen.blit(ucs_text, (ucs_button.x + 50, ucs_button.y + 10))

        pygame.display.flip()

    pygame.quit()


if __name__ == "__main__":
    main()

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.edges = {}
        self.isAnimal = False
    def connect(self, node, weight):
        self.edges[node] = weight

class Graph:
    def __init__(self, width, height, animals, obstacles):
        self.nodes = [[Node(x, y) for y in range(height)] for x in range(width)]
        self.width = width
        self.height = height

        for x in range(width):
            for y in range(height):
                weight = -1
                if x > 0:
                    self.nodes[x][y].connect(self.nodes[x - 1][y], weight)
                if x < width - 1:
                    self.nodes[x][y].connect(self.nodes[x + 1][y], weight)
                if y > 0:
                    self.nodes[x][y].connect(self.nodes[x][y - 1], weight)
                if y < height - 1:
                    self.nodes[x][y].connect(self.nodes[x][y + 1], weight)

        for animal in animals:
            self.nodes[animal[0]][animal[1]].isAnimal = True

        for obstacle in obstacles:
            self.nodes[obstacle[0][0]][obstacle[0][1]].edges[self.nodes[obstacle[1][0]][obstacle[1][1]]] = -3
            self.nodes[obstacle[1][0]][obstacle[1][1]].edges[self.nodes[obstacle[0][0]][obstacle[0][1]]] = -3

In [7]:
import random
animals = []
obstacles = []
for i in range(random.randint(3, 5)):
    animals.append((random.randint(0, 7), random.randint(0, 7)))
for i in range(random.randint(3, 5)):
    x = random.randint(0, 7)
    y = random.randint(0, 7)
    direction = random.choice(["up", "down", "left", "right"])
    if x == 0 and direction == "left":
        direction = "right"
    if x == 7 and direction == "right":
        direction = "left"
    if y == 0 and direction == "up":
        direction = "down"
    if y == 7 and direction == "down":
        direction = "up"
    if direction == "up":
        obstacles.append(((x, y), (x, y - 1)))
    elif direction == "down":
        obstacles.append(((x, y), (x, y + 1)))
    elif direction == "left":
        obstacles.append(((x, y), (x - 1, y)))
    elif direction == "right":
        obstacles.append(((x, y), (x + 1, y)))
    
print(obstacles)
graph = Graph(8, 8, animals, obstacles)

[((3, 5), (2, 5)), ((6, 7), (6, 6)), ((7, 4), (7, 5)), ((5, 3), (6, 3))]


In [8]:
%matplotlib qt
import networkx as nx
import matplotlib.pyplot as plt
def draw_graph(graph):
    G = nx.Graph()
    for x in range(graph.width):
        for y in range(graph.height):
            for node, weight in graph.nodes[x][y].edges.items():
                if not G.has_edge((x, y), (node.x, node.y)):
                    G.add_edge((x, y), (node.x, node.y), weight=-weight)

    path = nx.shortest_path(G, (0, 0), (7, 7), weight='weight')

    pos = {(x, y): (y, x) for x in range(graph.width) for y in range(graph.height)} 
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=1000, node_shape='s')
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

    path_edges = list(zip(path, path[1:]))
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='r', width=2)

    plt.axis('equal')
    plt.show()

draw_graph(graph)

In [41]:
import matplotlib.pyplot as plt
import networkx as nx
import random
from queue import PriorityQueue

class Node:
    def __init__(self, x, y, element):
        self.x = x
        self.y = y
        self.element = element  # 0: Safe, 1: Obstacle, 2: Start, 3: Goal
        self.edges = {}
    
    def connect(self, node, weight):
        self.edges[node] = weight

    # To make nodes comparable in PriorityQueue
    def __lt__(self, other):
        return False

class Graph:
    def __init__(self, width, height):
        self.nodes = [[None for _ in range(height)] for _ in range(width)]
        self.width = width
        self.height = height

        # Initialize nodes
        for x in range(width):
            for y in range(height):
                element = random.choice([0, 0, 0, 1])  # 75% chance of safe grid, 25% chance of obstacle
                if x == 0 and y == 0:
                    element = 2  # Start
                elif x == width - 1 and y == height - 1:
                    element = 3  # Goal
                self.nodes[x][y] = Node(x, y, element)

        # Connect nodes
        for x in range(width):
            for y in range(height):
                current_node = self.nodes[x][y]
                # Connect to adjacent nodes with appropriate cost
                for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:  # Right, down, left, up
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < width and 0 <= ny < height:
                        next_node = self.nodes[nx][ny]
                        weight = random.randint(-3, -1) if next_node.element in [0, 2, 3] else random.randint(-4, -2)
                        current_node.connect(next_node, weight)

def uniform_cost_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put((0, start, [start]))  # (cost, node, path)
    visited = set()
    
    while not frontier.empty():
        current_cost, current_node, path = frontier.get()
        
        if current_node == goal:
            return [(node.x, node.y) for node in path], -current_cost
        
        visited.add((current_node.x, current_node.y))
        
        for next_node, weight in current_node.edges.items():
            if (next_node.x, next_node.y) not in visited:
                new_cost = current_cost + weight
                frontier.put((new_cost, next_node, path + [next_node]))
                
    return None, float('inf')

def display_graph(graph, path):
    G = nx.DiGraph()
    pos = {}
    labels = {}
    for x in range(graph.width):
        for y in range(graph.height):
            node = graph.nodes[x][y]
            pos[node] = (y, -x)  # Invert x to display correctly
            G.add_node(node)
            if node.element == 2:  # Start
                labels[node] = 'S'
            elif node.element == 3:  # Goal
                labels[node] = 'G'
            elif node.element == 1:  # Obstacle
                labels[node] = 'O'
            else:
                labels[node] = ''

            for edge, weight in node.edges.items():
                G.add_edge(node, edge, weight=weight)

    # Convert path to edges for highlighting
    path_nodes = [graph.nodes[x][y] for x, y in path] if path else []
    edges_in_path = [(path_nodes[i], path_nodes[i+1]) for i in range(len(path_nodes) - 1)]
    
    # Draw the graph
    nx.draw(G, pos, labels=labels, with_labels=True, node_size=700, node_color='skyblue')
    nx.draw_networkx_edges(G, pos, edgelist=edges_in_path, edge_color='r', width=2)
    
    plt.title('Graph Traversal')
    plt.show()

# Example usage
graph = Graph(8, 8)  # Create a graph with specified dimensions
start_node = graph.nodes[0][0]  # Start node
goal_node = graph.nodes[7][7]  # Goal node
path, total_cost = uniform_cost_search(graph, start_node, goal_node)

print("Optimal Path:", path)
print("Total Cost:", total_cost)

if path:
    display_graph(graph, path)


Optimal Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (3, 3), (3, 2), (3, 1), (4, 1), (4, 2), (5, 2), (5, 1), (6, 1), (6, 2), (7, 2), (7, 3), (6, 3), (5, 3), (4, 3), (4, 4), (4, 5), (3, 5), (3, 6), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7)]
Total Cost: 79


In [36]:
import random

class Node:
    def __init__(self, x, y, element):
        self.x = x
        self.y = y
        self.element = element
        self.edges = {}
    
    def connect(self, node, weight):
        self.edges[node] = weight

class Graph:
    def __init__(self, width, height):
        self.nodes = [[None for _ in range(height)] for _ in range(width)]
        self.width = width
        self.height = height

        # Generate nodes for the graph
        for x in range(width):
            for y in range(height):
                # Randomly assign element type (0 for safegrid, 1 for obstacle)
                element = random.choice([0, 0, 0, 1])  # 75% chance of a safe grid, 25% chance of an obstacle
                
                # Assign start point (2) and goal point (3)
                if x == 0 and y == 0:
                    element = 2  # Start point
                elif x == width - 1 and y == height - 1:
                    element = 3  # Goal point
                
                # Create node with assigned element
                self.nodes[x][y] = Node(x, y, element)

        # Connect nodes with edges based on element type
        for x in range(width):
            for y in range(height):
                current_node = self.nodes[x][y]
                if current_node.element == 0:  # Safegrid
                    current_node.connect(self.nodes[x - 1][y], random.randint(-3, -1)) if x > 0 else None
                    current_node.connect(self.nodes[x + 1][y], random.randint(-3, -1)) if x < width - 1 else None
                    current_node.connect(self.nodes[x][y - 1], random.randint(-3, -1)) if y > 0 else None
                    current_node.connect(self.nodes[x][y + 1], random.randint(-3, -1)) if y < height - 1 else None
                elif current_node.element == 1:  # Obstacle
                    current_node.connect(self.nodes[x - 1][y], random.randint(-4, -2)) if x > 0 else None
                    current_node.connect(self.nodes[x + 1][y], random.randint(-4, -2)) if x < width - 1 else None
                    current_node.connect(self.nodes[x][y - 1], random.randint(-4, -2)) if y > 0 else None
                    current_node.connect(self.nodes[x][y + 1], random.randint(-4, -2)) if y < height - 1 else None

def uniform_cost_search(graph, start, goal):
    visited = set()
    frontier = [(0, start)]
    came_from = {}
    cost_so_far = {}
    came_from[start.x, start.y] = None  # Store coordinates instead of Node object
    cost_so_far[start.x, start.y] = 0

    while frontier:
        current_cost, current_node = min(frontier)
        frontier.remove((current_cost, current_node))

        if current_node == goal:
            break

        visited.add(current_node)

        for next_node, edge_cost in current_node.edges.items():
            new_cost = cost_so_far[current_node.x, current_node.y] + edge_cost  # Use coordinates for lookup
            if next_node not in cost_so_far or new_cost < cost_so_far[next_node.x, next_node.y]:
                cost_so_far[next_node.x, next_node.y] = new_cost
                priority = new_cost
                frontier.append((priority, next_node))
                came_from[next_node.x, next_node.y] = (current_node.x, current_node.y)  # Store coordinates

    # Reconstruct path
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = came_from.get((current.x, current.y), None)  # Look up coordinates
        if current is None:
            break  # Stop if current node is None
    path.append(start)
    path.reverse()

    return path, cost_so_far.get((goal.x, goal.y), float('inf'))  # Return infinity if goal node not reached

# Run Uniform Cost Search
start_node = graph.nodes[0][0]
goal_node = graph.nodes[7][7]  # Corrected indexing to (7, 7)
path, total_cost = uniform_cost_search(graph, start_node, goal_node)

# Output the optimal path and total cost
print("Optimal Path:")
for node in path:
    print(node.x, node.y)
print("Total Cost:", total_cost)


Optimal Path:
0 0
7 7
Total Cost: inf


In [1]:
%matplotlib qt
import networkx as nx
import matplotlib.pyplot as plt

class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.edges = {}
        self.isAnimal = False

    def connect(self, node, weight):
        self.edges[node] = weight

class Graph:
    def __init__(self, width, height, animals, obstacles):
        self.nodes = [[Node(x, y) for y in range(height)] for x in range(width)]
        self.width = width
        self.height = height

        for x in range(width):
            for y in range(height):
                weight = -1
                if x > 0:
                    self.nodes[x][y].connect(self.nodes[x - 1][y], weight)
                if x < width - 1:
                    self.nodes[x][y].connect(self.nodes[x + 1][y], weight)
                if y > 0:
                    self.nodes[x][y].connect(self.nodes[x][y - 1], weight)
                if y < height - 1:
                    self.nodes[x][y].connect(self.nodes[x][y + 1], weight)

        for animal in animals:
            self.nodes[animal[0]][animal[1]].isAnimal = True

        for obstacle in obstacles:
            self.nodes[obstacle[0][0]][obstacle[0][1]].edges[self.nodes[obstacle[1][0]][obstacle[1][1]]] = -3
            self.nodes[obstacle[1][0]][obstacle[1][1]].edges[self.nodes[obstacle[0][0]][obstacle[0][1]]] = -3

import random
animals = []
obstacles = []
for i in range(random.randint(3, 5)):
    animals.append((random.randint(0, 7), random.randint(0, 7)))
for i in range(random.randint(3, 5)):
    x = random.randint(0, 7)
    y = random.randint(0, 7)
    direction = random.choice(["up", "down", "left", "right"])
    if x == 0 and direction == "left":
        direction = "right"
    if x == 7 and direction == "right":
        direction = "left"
    if y == 0 and direction == "up":
        direction = "down"
    if y == 7 and direction == "down":
        direction = "up"
    if direction == "up" and y > 0:
        obstacles.append(((x, y), (x, y - 1)))
    elif direction == "down" and y < 7:
        obstacles.append(((x, y), (x, y + 1)))
    elif direction == "left" and x > 0:
        obstacles.append(((x, y), (x - 1, y)))
    elif direction == "right" and x < 7:
        obstacles.append(((x, y), (x + 1, y)))

print(obstacles)
graph = Graph(8, 8, animals, obstacles)


def draw_graph(graph):
    G = nx.Graph()
    for x in range(graph.width):
        for y in range(graph.height):
            for node, weight in graph.nodes[x][y].edges.items():
                if not G.has_edge((x, y), (node.x, node.y)):
                    G.add_edge((x, y), (node.x, node.y), weight=-weight)

    path = nx.shortest_path(G, (0, 0), (7, 7), weight='weight')

    pos = {(x, y): (y, x) for x in range(graph.width) for y in range(graph.height)} 
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=1000, node_shape='s')
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

    path_edges = list(zip(path, path[1:]))
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='r', width=2)

    plt.axis('equal')
    plt.show()

draw_graph(graph)


[((6, 7), (6, 6)), ((2, 1), (1, 1)), ((6, 3), (6, 2))]


In [19]:
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.edges = {}
        self.isAnimal = False
    
    def connect(self, node, weight):
        self.edges[node] = weight

class Graph:
    def __init__(self, width, height, animals, obstacles):
        self.nodes = [[Node(x, y) for y in range(height)] for x in range(width)]
        self.width = width
        self.height = height

        for x in range(width):
            for y in range(height):
                weight = -1
                if x > 0:
                    self.nodes[x][y].connect(self.nodes[x - 1][y], weight)
                if x < width - 1:
                    self.nodes[x][y].connect(self.nodes[x + 1][y], weight)
                if y > 0:
                    self.nodes[x][y].connect(self.nodes[x][y - 1], weight)
                if y < height - 1:
                    self.nodes[x][y].connect(self.nodes[x][y + 1], weight)

        for animal in animals:
            self.nodes[animal[0]][animal[1]].isAnimal = True

        for obstacle in obstacles:
            # Adjust weight for obstacle
            weight = -3  # Change this as needed
            self.nodes[obstacle[0][0]][obstacle[0][1]].edges[self.nodes[obstacle[1][0]][obstacle[1][1]]] = weight
            self.nodes[obstacle[1][0]][obstacle[1][1]].edges[self.nodes[obstacle[0][0]][obstacle[0][1]]] = weight

        for x in range(width):
            for y in range(height):
                if self.nodes[x][y].isAnimal:
                    # Adjust weight for animal encounter
                    weight = random.randint(-4, -2)  # Random cost between -4 and -2
                    for neighbor in self.nodes[x][y].edges:
                        self.nodes[x][y].edges[neighbor] = weight
                        neighbor.edges[self.nodes[x][y]] = weight

import random

animals = []
obstacles = []

# Randomly generate animals and obstacles
for i in range(random.randint(3, 5)):
    animals.append((random.randint(0, 7), random.randint(0, 7)))

for i in range(random.randint(3, 5)):
    x = random.randint(0, 7)
    y = random.randint(0, 7)
    direction = random.choice(["up", "down", "left", "right"])
    if x == 0 and direction == "left":
        direction = "right"
    if x == 7 and direction == "right":
        direction = "left"
    if y == 0 and direction == "up":
        direction = "down"
    if y == 7 and direction == "down":
        direction = "up"
    if direction == "up":
        obstacles.append(((x, y), (x, y - 1)))
    elif direction == "down":
        obstacles.append(((x, y), (x, y + 1)))
    elif direction == "left":
        obstacles.append(((x, y), (x - 1, y)))
    elif direction == "right":
        obstacles.append(((x, y), (x + 1, y)))

# Print obstacles for verification
print(obstacles)

# Create graph with animals and obstacles
graph = Graph(8, 8, animals, obstacles)


[((4, 3), (4, 2)), ((0, 2), (1, 2)), ((6, 1), (5, 1)), ((4, 7), (4, 6))]


In [1]:
%matplotlib qt
import networkx as nx
import matplotlib.pyplot as plt

class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.edges = {}
        self.isAnimal = False
    
    def connect(self, node, weight):
        self.edges[node] = weight

class Graph:
    def __init__(self, width, height, animals, obstacles):
        self.nodes = [[Node(x, y) for y in range(height)] for x in range(width)]
        self.width = width
        self.height = height

        for x in range(width):
            for y in range(height):
                weight = -1
                if x > 0:
                    self.nodes[x][y].connect(self.nodes[x - 1][y], weight)
                if x < width - 1:
                    self.nodes[x][y].connect(self.nodes[x + 1][y], weight)
                if y > 0:
                    self.nodes[x][y].connect(self.nodes[x][y - 1], weight)
                if y < height - 1:
                    self.nodes[x][y].connect(self.nodes[x][y + 1], weight)

        for animal in animals:
            self.nodes[animal[0]][animal[1]].isAnimal = True

        for obstacle in obstacles:
            weight = -3  # Change this as needed
            self.nodes[obstacle[0][0]][obstacle[0][1]].edges[self.nodes[obstacle[1][0]][obstacle[1][1]]] = weight
            self.nodes[obstacle[1][0]][obstacle[1][1]].edges[self.nodes[obstacle[0][0]][obstacle[0][1]]] = weight

        for x in range(width):
            for y in range(height):
                if self.nodes[x][y].isAnimal:
                    weight = random.randint(-4, -2)  # Random cost between -4 and -2
                    for neighbor in self.nodes[x][y].edges:
                        self.nodes[x][y].edges[neighbor] = weight
                        neighbor.edges[self.nodes[x][y]] = weight

def draw_graph(graph):
    G = nx.Graph()
    for x in range(graph.width):
        for y in range(graph.height):
            for node, weight in graph.nodes[x][y].edges.items():
                if not G.has_edge((x, y), (node.x, node.y)):
                    G.add_edge((x, y), (node.x, node.y), weight=-weight)

    path = nx.shortest_path(G, (0, 0), (7, 7), weight='weight')

    pos = {(x, y): (y, x) for x in range(graph.width) for y in range(graph.height)} 
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=1000, node_shape='s')
    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

    path_edges = list(zip(path, path[1:]))
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='r', width=2)

    plt.axis('equal')
    plt.show()

# Assuming 'graph' is already defined
draw_graph(graph)


NameError: name 'graph' is not defined